In [1]:
import os
import dask
from dask.diagnostics import ProgressBar
import numpy as np
import my_funs
import xarray as xr
import cProfile
import zarr
import matplotlib.pylab as plt
import geopandas as gpd
import folium
from folium.plugins import MousePosition
import pandas as pd
# from dask.distributed import Client, progress
# client = Client(processes=False)
dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/"

/data/home/hamiddashti/hamid/nasa_above/greeness/codes/src/my_funs.py:9: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


Filtering the LAI and NDVI data based on quality control (LAI data from 1984-2014)

In [ ]:
ds_lai = xr.open_zarr(
    dir + "data/raw_data/noaa_cdr/lai_fapar/noaa_lai_fapar_clipped_raw_1984_2013.zarr"
)
lai = ds_lai["LAI"]
fapar = ds_lai["FAPAR"]
qa_lai = ds_lai["QA"]
qa_lai_binary = my_funs.dec2bin(qa_lai, 9)
qa_lai_binary = qa_lai_binary.astype(str)
qa_lai_mask = my_funs.avhrr_mask(qa_lai_binary, dask="allowed", var="LAI")
lai_masked = lai.where(qa_lai_mask)
lai_masked = lai_masked.chunk({"time": 100})

# This will take hourse to run. 
with ProgressBar():
    lai_filtered = lai_masked.compute()
lai_filtered.to_netcdf(dir+"data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered_1984_2013.nc")

we later downloaded the 2013 to 2021 as well. Below are for filtering those.

In [ ]:
ds_lai = xr.open_zarr(
    dir + "data/raw_data/noaa_cdr/lai_fapar/noaa_lai_fapar_clipped_raw_2014_2021.zarr"
)
lai = ds_lai["LAI"]
fapar = ds_lai["FAPAR"]
qa_lai = ds_lai["QA"]
qa_lai_binary = my_funs.dec2bin(qa_lai, 9)
qa_lai_binary = qa_lai_binary.astype(str)
# qa_lai_mask = my_funs.avhrr_mask(qa_lai_binary, dask="allowed", var="LAI")
qa_lai_mask = my_funs.avhrr_mask(qa_lai_binary, dask="allowed", var="LAI")
lai_masked = lai.where(qa_lai_mask)
lai_masked = lai_masked.chunk({"time": 100})
# lai_maked_ds = lai_masked.to_dataset()
# m = client.compute(lai_maked_ds)
# progress(m)
with ProgressBar():
    lai_filtered_2014_2021 = lai_masked.compute()
lai_filtered_2014_2021.to_netcdf(dir+"data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered_2014_2021.nc")

Rest is code for FPAR and ndvi if needed. 

In [ ]:
# ds_ndvi = xr.open_zarr(dir + "data/raw_data/noaa_cdr/ndvi/noaa_ndvi_clipped_raw.zarr")
# ndvi = ds_ndvi["NDVI"]
# qa_ndvi = ds_ndvi["QA"]
# # Convert QAs to binary
# qa_ndvi_binary = my_funs.dec2bin(qa_ndvi, 16)
# fapar_maked = fapar.where(qa_lai_mask)
# qa_ndvi_binary = qa_ndvi_binary.astype(str)
# qa_mask_ndvi = my_funs.avhrr_mask(qa_ndvi_binary, dask="allowed", var="NDVI")
# ndvi_masked = ndvi.where(qa_mask_ndvi)


In [ ]:
# with ProgressBar():
#     lai_filtered = lai_masked.compute()
# with ProgressBar():
#     ndvi_filtered = ndvi_masked.compute()

# In case staritng the dask distributed client the above progressbar does not work and we should use the following commands
# lai_maked_ds = lai_masked.to_dataset()
# fapar_maked_ds = fapar_maked.to_dataset()
# m = client.compute(lai_maked_ds)
# progress(m)
# ndvi_masked_ds = ndvi_masked.to_dataset()
# m2 = client.compute(ndvi_masked_ds)


In [ ]:
# fapar_maked_ds.to_zarr(
#     out_dir + "data/processed_data/noaa_nc/lai_fapar/filtered/fapar_filtered.zarr"
# )

In [ ]:
# progress(m)
# fapar_maked_ds.to_zarr(
#     out_dir + "data/processed_data/noaa_nc/lai_fapar/filtered/fapar_filtered.zarr"
# )


In [ ]:
# progress(m2)
# ndvi_masked_ds.to_zarr(
#     out_dir + "data/processed_data/noaa_nc/ndvi/filtered/ndvi_filtered.zarr"
# )


Resample daily LAI to annual

In [6]:
# Resampling LAI 1984 to 2013
lai = xr.open_dataarray(
    dir + "/data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered_1984_2013.nc"
)
lai_annual_resample_max = lai.groupby("time.year").max().rename({"year":"time"})
lai_annual_resample_max['time'] = pd.date_range("1984","2014",freq = "A")
lai_annual_resample_max.to_netcdf(
     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max_1984_2013.nc"
 )

# Resampling LAI 2014 to 2021
lai = xr.open_dataarray(
    dir + "/data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered_2014_2021.nc"
)
lai_annual_resample_max = lai.groupby("time.year").max().rename({"year":"time"})
lai_annual_resample_max['time'] = pd.date_range("2014","2022",freq = "A")
lai_annual_resample_max.to_netcdf(
     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max_2014_2021.nc"
 )

In [2]:
lai = xr.open_dataarray(
    dir + "/data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered_1984_2013.nc"
)
lai_monthly_resample_mean = lai.resample(time="1MS").mean()
lai_summer_resample_mean = lai_monthly_resample_mean.where(
    lai_monthly_resample_mean.time.dt.month.isin([6, 7, 8]), drop=True
)
lai_summer_resample_mean = lai_summer_resample_mean.rename({"latitude":"lat","longitude":"lon"})
lai_summer_resample_mean.to_netcdf(dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_summer_resample_mean.nc")

<xarray.DataArray 'LAI' (time: 90, latitude: 448, longitude: 1348)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * latitude     (latitude) float32 74.53 74.47 74.43 ... 52.27 52.22 52.17
  * longitude    (longitude) float32 -168.1 -168.0 -168.0 ... -100.8 -100.7
    spatial_ref  int64 0
  * time         (time) datetime64[ns] 1984-06-01 1984-07-01 ... 2013-08-01
Attributes:
    grid_mapping:   crs
    long_name:      Leaf Area Index
    standard_name:  leaf_area_index
    units:          1
    valid_range:    [   0 7000]

Below is for resampling to other time windows in case needed. 

In [ ]:
# lai = xr.open_dataarray(
#     dir + "/data/processed_data/noaa_nc/lai_fapar/filtered/lai_filtered.nc"
# )

# lai_monthly_resample_mean = lai.resample(time="1MS").mean()
# lai_monthly_resample_mean.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_resample_mean.nc"
# )

# lai_growing_monthly_resample_mean = lai_monthly_resample_mean.where(
#     lai_monthly_resample_mean.time.dt.month.isin([5, 6, 7, 8, 9, 10]), drop=True
# )
# lai_growing_monthly_resample_mean.to_netcdf(
#     dir
#     + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing_monthly_resample_mean.nc"
# )
# print("lai_monthly_resample_mean Done!")

# lai_monthly_resample_max = lai.resample(time="1MS").max()
# lai_monthly_resample_max.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_resample_max.nc"
# )
# print("lai_monthly_resample_max Done!")

# lai_monthly_group_mean = lai.groupby("time.month").mean()
# lai_monthly_group_mean.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_group_mean.nc"
# )
# print("lai_monthly_group_mean Done!")

# lai_monthly_group_max = lai.groupby("time.month").max()
# lai_monthly_group_max.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_group_max.nc"
# )
# print("lai_monthly_group_max Done!")

# lai_seasonal_resample_mean = my_funs.weighted_season_resmaple(lai)
# lai_seasonal_resample_mean.to_netcdf(
#     dir
#     + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_seasonal_resample_mean.nc"
# )
# print("lai_seasonal_resample_mean Done!")

# lai_seaonal_group_mean = my_funs.weighted_season_group(lai)
# lai_seaonal_group_mean.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_seaonal_group_mean.nc"
# )
# print("lai_seaonal_group_mean Done!")

# lai_seasonal_resample_max = lai.resample(time="Q-DEC").max()
# lai_seasonal_resample_max.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_seasonal_resample_max.nc"
# )
# print("lai_seasonal_resample_max Done!")

# lai_seasonal_group_max = lai.groupby("time.season").max()
# lai_seasonal_group_max.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_seasonal_group_max.nc"
# )
# print("lai_seasonal_group_max Done!")

# lai_annualresample_mean = lai.groupby("time.year").mean()
# lai_annualresample_mean.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_mean.nc"
# )
# print("lai_annualresample_mean Done!")

# lai_annual_resample_max = lai.groupby("time.year").max()
# lai_annual_resample_max.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max.nc"
# )
# lai_annual_resample_max['time'] = pd.date_range("1984","2014",freq = "A")
# print("lai_annual_resample_max Done!")

# lai_growing = my_funs.growing_season(lai)
# lai_growing.to_netcdf(
#     dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing_mean.nc"
# )
# print("All done!")
